In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# detrend + standardization

In [ ]:
! pip install pyts
from pyts.decomposition import SingularSpectrumAnalysis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 37.9 MB/s eta 0:00:00


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# set start year
sy = 1981

## read climate data

In [ ]:
df_x_ex = pd.read_csv("df_x_ex.csv", index_col=0)
df_x_ex['code'] = df_x_ex_1['code'].astype(str).str.rjust(5,'0')
df_x_ex.head()

,code,year,dtr,frs,tmp,pre,TXx,TNn,tx90p,tn10p,rx5day,spi6,spei6
0,01001,1982,-0.066827,0.081247,-0.409364,0.458933,-0.431328,-0.738092,-0.335976,0.214563,0.670635,0.686287,0.712975
1,01003,1982,0.963552,-0.030733,0.023913,-0.767014,0.241740,-1.487991,0.773903,0.424831,-0.688480,-1.337725,-1.396091
2,01005,1982,-0.014431,0.268783,-0.401021,0.250536,-0.114096,-1.117583,-0.047581,0.270809,-0.398192,0.533390,-0.271566
3,01007,1982,1.084958,0.080535,-0.485869,0.161403,-0.156288,-1.879778,-0.223870,0.162030,0.204651,0.201990,0.162277
4,01009,1982,0.284645,-0.110597,-0.434435,0.767210,-0.235106,-0.668280,-0.239965,0.019044,0.561905,-0.136115,-0.186780


# read agricultural data
list the data to be one line.
1. harvestable fraction
2. yield
3. production
4. planted area of the next year
5. planted area of this year

state, county, year, HF, yield, Prod, PA_next, PA_this

In [ ]:
# extract the county code
cc = pd.read_csv("har.csv")
cc['state'] = cc['State ANSI'].astype(str).str.rjust(2,'0')
cc['county'] = cc['County ANSI'].astype(str).str.rjust(3,'0')
cc.head()

In [ ]:
# input filenames
fname = ["ratio_3.csv",
          "0_yield_fmt.csv",
          "0_prod_fmt.csv",
          "0_pla_fmt.csv"]

# output filenames
sname_dtst = ["hf_dtst.csv",
         "yid_dtst.csv",
         'prod_dtst.csv',
         'pla_dtst.csv']

# variable names
vname2 = ['ratio','yield','prod','pla']

In [ ]:
# output dictionary
oname2 = {}

## detrend & standardize

In [ ]:
ratio_fmt = pd.read_csv(fname[0])
for j in range(4):
  vb0 = pd.read_csv(fname[j])
  # -- filter useless rows
  vb0 = ratio_fmt[['State','County']].merge(vb0, how='left')

  vb = vb0.drop(columns=['State', 'County'])
  vb_t = vb.copy().T

  # -- detrend  ####
  vb_ny = vb.to_numpy()

  for i in range(len(vb)):
    # i=0
    to_dt=vb_ny[i,:][~np.isnan(vb_ny[i])].reshape(1,-1)
    # Singular Spectrum Analysis
    ssa = SingularSpectrumAnalysis(window_size=10)
    vb_ssa = ssa.fit_transform(to_dt)
    back_value = to_dt[0] - vb_ssa[0]
    # find the index of non-NA rows
    slt_ind = vb_t[i].index[~vb_t[i].isnull()]
    vb_t.loc[slt_ind,i] = back_value

  # -- tranpose back to normal format
  vb_to_st = vb_t.copy().T

  # -- standardize ####
  vb_st=vb_to_st.sub(vb_to_st.mean(1), axis=0).div(vb_to_st.std(1), axis=0)
  vb_st[['State', 'County']] = vb0[['State', 'County']]

  # -- format the final rows to merge
  vb_to_m = vb_st.melt(id_vars=['State','County'])
  vb_to_m = vb_to_m.rename(columns={'variable': 'year', 'value': vname[j]})

  vb_to_m = vb_to_m.dropna()
  vb_to_m["year"]= vb_to_m["year"].str[-4:]
  vb_to_m['year'] =vb_to_m['year'].astype(int)
  vb_one = vb_to_m.merge(cc[['State',	'state',	'County',	'county']],on=['State','County']).drop_duplicates()

  oname[j] = vb_one.copy()
  oname[j].to_csv(sname_dtst[j])

